In [1]:
import tensorflow as tf
import os

BASE_PATH = "/kaggle/input/datasets/sapampritamjit/real-image-tomato-dataset/Variant-a(Multiclass Classification)"

train_dir = os.path.join(BASE_PATH, "train")
val_dir   = os.path.join(BASE_PATH, "val")

IMG_SIZE = 300
BATCH_SIZE = 32


2026-02-15 02:09:16.387433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771121356.565529      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771121356.616241      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771121357.035723      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771121357.035766      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771121357.035769      55 computation_placer.cc:177] computation placer alr

In [2]:
import numpy as np

from tensorflow.keras.applications.efficientnet import preprocess_input

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print("Classes:", class_names)


Found 3162 files belonging to 8 classes.


I0000 00:00:1771121373.448843      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13757 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1771121373.454443      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13757 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 902 files belonging to 8 classes.
Classes: ['Early_blight', 'Healthy', 'Late_blight', 'Leaf Miner', 'Magnesium Deficiency', 'Nitrogen Deficiency', 'Pottassium Deficiency', 'Spotted Wilt Virus']


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)


In [5]:
IMG_SIZE = 300
num_classes = 8  # real dataset has 8 classes

In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomBrightness(0.2),
])

In [7]:
base_model = tf.keras.applications.EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  # Freeze initially

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = data_augmentation(inputs)          # 🔥 NEW
x = preprocess_input(x)
x = base_model(x, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=3
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20


E0000 00:00:1771121394.921082      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1771121398.373513     127 cuda_dnn.cc:529] Loaded cuDNN version 91002


99/99 ━━━━━━━━━━━━━━━━━━━━ 57s 370ms/step - accuracy: 0.2602 - loss: 1.9806 - val_accuracy: 0.4446 - val_loss: 1.7148 - learning_rate: 1.0000e-04
Epoch 2/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 31s 318ms/step - accuracy: 0.4147 - loss: 1.6965 - val_accuracy: 0.4978 - val_loss: 1.5801 - learning_rate: 1.0000e-04
Epoch 3/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 32s 324ms/step - accuracy: 0.4435 - loss: 1.5701 - val_accuracy: 0.5122 - val_loss: 1.4909 - learning_rate: 1.0000e-04
Epoch 4/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 32s 319ms/step - accuracy: 0.5031 - loss: 1.4753 - val_accuracy: 0.5233 - val_loss: 1.4281 - learning_rate: 1.0000e-04
Epoch 5/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 32s 320ms/step - accuracy: 0.5084 - loss: 1.4208 - val_accuracy: 0.5333 - val_loss: 1.3779 - learning_rate: 1.0000e-04
Epoch 6/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 32s 321ms/step - accuracy: 0.5304 - loss: 1.3457 - val_accuracy: 0.5410 - val_loss: 1.3363 - learning_rate: 1.0000e-04
Epoch 7/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 32s 320ms/step - accuracy: 0.55

In [10]:
base_model.trainable = True

model.compile(
    optimizer = tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)


Epoch 1/20


E0000 00:00:1771122094.064500      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


99/99 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.3404 - loss: 1.7604 - val_accuracy: 0.6408 - val_loss: 1.0693
Epoch 2/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.5392 - loss: 1.3707 - val_accuracy: 0.6419 - val_loss: 1.0265
Epoch 3/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.5977 - loss: 1.1821 - val_accuracy: 0.6619 - val_loss: 0.9632
Epoch 4/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.6374 - loss: 1.0435 - val_accuracy: 0.6785 - val_loss: 0.9085
Epoch 5/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.6810 - loss: 0.9502 - val_accuracy: 0.6962 - val_loss: 0.8539
Epoch 6/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.7051 - loss: 0.8782 - val_accuracy: 0.7073 - val_loss: 0.8110
Epoch 7/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.7018 - loss: 0.8444 - val_accuracy: 0.7217 - val_loss: 0.7710
Epoch 8/20
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.7237 - loss: 0.8107 - val_accuracy: 0.7339 - val_loss: 0.

In [11]:
# Lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_finetune2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10


E0000 00:00:1771124576.978866      55 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


99/99 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.8636 - loss: 0.4137 - val_accuracy: 0.8537 - val_loss: 0.4379
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8568 - loss: 0.4112 - val_accuracy: 0.8581 - val_loss: 0.4276
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8687 - loss: 0.3917 - val_accuracy: 0.8681 - val_loss: 0.4083
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8653 - loss: 0.3784 - val_accuracy: 0.8725 - val_loss: 0.4017
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8797 - loss: 0.3590 - val_accuracy: 0.8780 - val_loss: 0.3880
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8853 - loss: 0.3375 - val_accuracy: 0.8825 - val_loss: 0.3792
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8905 - loss: 0.3248 - val_accuracy: 0.8836 - val_loss: 0.3718
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8885 - loss: 0.2979 - val_accuracy: 0.8869 - val_loss: 0.

In [12]:
model.save("variant_a_final.keras")

# Variant - B

In [22]:
import pandas as pd
import tensorflow as tf
import os

IMG_SIZE = 300
BATCH_SIZE = 32
NUM_CLASSES = 8

BASE_PATH_B = "/kaggle/input/datasets/sapampritamjit/real-image-tomato-dataset/Variant-b(MultiLabel Classification)"

csv_path = os.path.join(BASE_PATH_B, "/kaggle/input/datasets/sapampritamjit/real-image-tomato-dataset/Variant-b(MultiLabel Classification)/Multi-Label dataset - with augmented.csv")
images_dir = os.path.join(BASE_PATH_B, "images")

df = pd.read_csv(csv_path)

print(df.head())

  dataset                 filename                path  Early blight  Healthy  \
0    test  IMG20220323100545_1.jpg  \test\Early_blight           1.0      NaN   
1    test  IMG20220323102226_1.jpg  \test\Early_blight           1.0      NaN   
2    test  IMG20220323102423_7.jpg  \test\Early_blight           1.0      NaN   
3    test  IMG20220323102851_4.jpg  \test\Early_blight           1.0      NaN   
4    test  IMG20220323103905_4.jpg  \test\Early_blight           1.0      NaN   

   Late blight  Leaf Miner  Magnesium Deficiency  Nitrogen Deficiency  \
0          NaN         NaN                   NaN                  NaN   
1          NaN         NaN                   NaN                  NaN   
2          NaN         NaN                   NaN                  NaN   
3          NaN         NaN                   NaN                  NaN   
4          NaN         NaN                   NaN                  NaN   

   Pottassium Deficiency  Spotted Wilt Virus  SUM  
0                    N

In [23]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input

IMG_SIZE = 300
BATCH_SIZE = 16
NUM_CLASSES = 8

BASE_PATH_B = "/kaggle/input/datasets/sapampritamjit/real-image-tomato-dataset/Variant-b(MultiLabel Classification)"

csv_path = os.path.join(BASE_PATH_B, "Multi-Label dataset - with augmented.csv")

df = pd.read_csv(csv_path)

In [24]:
# Replace NaN with 0
df = df.fillna(0)

# Select disease columns (exact names from your CSV)
label_columns = [
    "Early blight",
    "Healthy",
    "Late blight",
    "Leaf Miner",
    "Magnesium Deficiency",
    "Nitrogen Deficiency",
    "Pottassium Deficiency",
    "Spotted Wilt Virus"
]

# Convert to int
df[label_columns] = df[label_columns].astype(int)

In [25]:
# Fix slashes
df["path"] = df["path"].str.replace("\\", "/", regex=False)

# Remove leading slash
df["path"] = df["path"].str.lstrip("/")

# Build correct full path (IGNORE 'path' column)
df["full_path"] = df.apply(
    lambda row: os.path.join(
        BASE_PATH_B,
        row["dataset"],     # train or test
        row["filename"]
    ),
    axis=1
)

train_df = df[df["dataset"] == "train"]
val_df   = df[df["dataset"] == "test"]

print("Train size:", len(train_df))
print("Val size:", len(val_df))


Train size: 3945
Val size: 584


In [26]:
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    return image, label

def make_dataset(dataframe, training=True):
    paths = dataframe["full_path"].values
    labels = dataframe[label_columns].values
    
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    
    if training:
        ds = ds.shuffle(buffer_size=len(dataframe))
    
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds_b = make_dataset(train_df, training=True)
val_ds_b   = make_dataset(val_df, training=False)

In [27]:
model_a = tf.keras.models.load_model("variant_a_final.keras")

base_model = model_a.get_layer("efficientnetb3")
base_model.trainable = False  # 🔥 Freeze backbone

In [28]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.4)(x)

outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)

model_b = tf.keras.Model(inputs, outputs)

model_b.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),  # slightly higher LR for head
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

In [29]:
for images, labels in train_ds_b.take(1):
    print(labels[0])

tf.Tensor([0 0 0 1 0 0 0 0], shape=(8,), dtype=int64)


In [30]:
history_phase1 = model_b.fit(
    train_ds_b,
    validation_data=val_ds_b,
    epochs=5
)

Epoch 1/5
247/247 ━━━━━━━━━━━━━━━━━━━━ 89s 220ms/step - binary_accuracy: 0.7066 - loss: 0.5613 - precision_1: 0.2848 - recall_1: 0.5208 - val_binary_accuracy: 0.9043 - val_loss: 0.3102 - val_precision_1: 0.7973 - val_recall_1: 0.4979
Epoch 2/5
247/247 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - binary_accuracy: 0.9028 - loss: 0.2871 - precision_1: 0.7858 - recall_1: 0.4956 - val_binary_accuracy: 0.9165 - val_loss: 0.2425 - val_precision_1: 0.8141 - val_recall_1: 0.5851
Epoch 3/5
247/247 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - binary_accuracy: 0.9175 - loss: 0.2273 - precision_1: 0.8186 - recall_1: 0.5748 - val_binary_accuracy: 0.9229 - val_loss: 0.2150 - val_precision_1: 0.8140 - val_recall_1: 0.6399
Epoch 4/5
247/247 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - binary_accuracy: 0.9249 - loss: 0.2063 - precision_1: 0.8273 - recall_1: 0.6345 - val_binary_accuracy: 0.9257 - val_loss: 0.2000 - val_precision_1: 0.8127 - val_recall_1: 0.6653
Epoch 5/5
247/247 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - binary_accur

In [31]:
base_model.trainable = True
model_b.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # small LR
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

In [32]:
history_phase2 = model_b.fit(
    train_ds_b,
    validation_data=val_ds_b,
    epochs=10
)


Epoch 1/10


2026-02-15 03:29:46.909781: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:29:47.062181: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:29:54.033096: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:29:54.178242: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:29:54.796763: E external/local_xla/xla/stream_

246/247 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - binary_accuracy: 0.9266 - loss: 0.1879 - precision_2: 0.8188 - recall_2: 0.6579

2026-02-15 03:31:49.026977: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:31:49.169550: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:31:54.240596: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:31:54.380516: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-02-15 03:31:54.886220: E external/local_xla/xla/stream_

247/247 ━━━━━━━━━━━━━━━━━━━━ 261s 566ms/step - binary_accuracy: 0.9266 - loss: 0.1878 - precision_2: 0.8189 - recall_2: 0.6579 - val_binary_accuracy: 0.9426 - val_loss: 0.1639 - val_precision_2: 0.8434 - val_recall_2: 0.7651
Epoch 2/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 62s 249ms/step - binary_accuracy: 0.9378 - loss: 0.1576 - precision_2: 0.8493 - recall_2: 0.7112 - val_binary_accuracy: 0.9463 - val_loss: 0.1500 - val_precision_2: 0.8433 - val_recall_2: 0.7947
Epoch 3/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 61s 246ms/step - binary_accuracy: 0.9447 - loss: 0.1422 - precision_2: 0.8631 - recall_2: 0.7503 - val_binary_accuracy: 0.9497 - val_loss: 0.1399 - val_precision_2: 0.8449 - val_recall_2: 0.8200
Epoch 4/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 61s 247ms/step - binary_accuracy: 0.9499 - loss: 0.1311 - precision_2: 0.8777 - recall_2: 0.7768 - val_binary_accuracy: 0.9533 - val_loss: 0.1314 - val_precision_2: 0.8588 - val_recall_2: 0.8298
Epoch 5/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 61s 247ms/step - binary_accura

In [33]:
import numpy as np

# Collect predictions
y_true = []
y_pred = []

for images, labels in val_ds_b:
    preds = model_b.predict(images)
    y_true.append(labels.numpy())
    y_pred.append(preds)

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [36]:
threshold = 0.4  # try 0.3, 0.4, 0.5

y_pred_binary = (y_pred > threshold).astype(int)

In [38]:
import numpy as np
from sklearn.metrics import classification_report

y_true = []
y_pred = []

for images, labels in val_ds_b:
    preds = model_b.predict(images)
    y_true.append(labels.numpy())
    y_pred.append(preds)

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

# Default threshold 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

print(classification_report(y_true, y_pred_binary, target_names=label_columns))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [80]:
threshold = 0.35   # try 0.3–0.4

y_pred_binary = (y_pred > threshold).astype(int)

print(classification_report(y_true, y_pred_binary, target_names=label_columns))

                       precision    recall  f1-score   support

         Early blight       0.83      0.82      0.83        61
              Healthy       0.75      0.82      0.78        22
          Late blight       0.86      0.97      0.91        95
           Leaf Miner       0.83      0.93      0.88       219
 Magnesium Deficiency       0.89      0.94      0.91       158
  Nitrogen Deficiency       0.78      0.90      0.83        50
Pottassium Deficiency       0.75      0.60      0.67        10
   Spotted Wilt Virus       0.86      0.83      0.85        96

            micro avg       0.84      0.90      0.87       711
            macro avg       0.82      0.85      0.83       711
         weighted avg       0.85      0.90      0.87       711
          samples avg       0.88      0.93      0.89       711



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image

IMG_SIZE = 300
THRESHOLD = 0.40
TOP_K = 3

def predict_image(img_path, model, label_columns):
    # Load image
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    preds = model.predict(img_array, verbose=0)[0]

    # Sort predictions (highest first)
    sorted_indices = np.argsort(preds)[::-1]

    results = []

    # Add top-k predictions above threshold
    for idx in sorted_indices[:TOP_K]:
        if preds[idx] >= THRESHOLD:
            results.append((label_columns[idx], float(preds[idx])))

    # If nothing passed threshold → return highest prediction anyway
    if len(results) == 0:
        best_idx = sorted_indices[0]
        results.append((label_columns[best_idx], float(preds[best_idx])))

    return results

In [91]:
predict_image("/kaggle/input/datasets/sapampritamjit/real-image-tomato-dataset/Variant-c(Object Detection)/train/images/IMG20220323083451_aug2.jpg",  model_b, label_columns)

[('Healthy', 0.6883466839790344),
 ('Pottassium Deficiency', 0.5590724945068359),
 ('Magnesium Deficiency', 0.5508484840393066)]

In [92]:
model_b.save("tomato_multilabel_final.keras")

In [93]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_b)
tflite_model = converter.convert()

with open("tomato_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpcnxhktq7/assets


INFO:tensorflow:Assets written to: /tmp/tmpcnxhktq7/assets


Saved artifact at '/tmp/tmpcnxhktq7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name='keras_tensor_2078')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  132162727871504: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  132162727882448: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  132169296012560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296011600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296010832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296013328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296013904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296011216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296010448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132169296013712: TensorSpec(shape=(), dtype=tf.resource, name

W0000 00:00:1771129801.528070      55 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1771129801.528112      55 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1771129801.892177      55 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


In [94]:
print(model_b.output_shape)
print(model_a.output_shape)

(None, 8)
(None, 8)


# Variance - c